# Folium Test 16/17

In [8]:
# Imports
import pandas as pd
import functions as fc
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster

sales_frame = fc.read_csv("data/nyc-rolling-sales2.csv")
zipcodes = sales_frame[sales_frame["ZIP CODE"]>0]
zipcodes["ZIP"] = zipcodes["ZIP CODE"].astype(str)
boroughs = zipcodes[["ZIP","BOROUGH"]]
boroughs.drop_duplicates("ZIP", inplace=True)
'''
from geopy.geocoders import Nominatim # get longitude and latitude based on the address
def get_lonlat(str_):
     geolocator = Nominatim(user_agent="abc")
     location = geolocator.geocode(str_, country_codes="US")
     try:
         return location.latitude, location.longitude
     except:
         return np.nan, np.nan

import requests
response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=1600+Amphitheatre+Parkway,+Mountain+View,+CA')
resp_json_payload = response.json()
print(resp_json_payload)

# too many requests
lonlat = []
for val in sales_frame['ADDRESS']:
     locatn = get_lonlat(val)
     print(val, locatn)
     lonlat.append(locatn)
lonlat=pd.DataFrame(lonlat, columns=["lon","lat"])
lonlat.to_csv(path_or_buf="data/lonlat.csv",index=False)
print("saved")
'''
us_zipcodes = pd.read_csv("data/US Zip Codes from 2013 Government Data.txt", delimiter= ",", dtype=str)
zipcodes_agg=pd.merge(zipcodes.groupby("ZIP").agg(np.mean), us_zipcodes, how="left", on="ZIP")
zipcodes_agg = pd.merge(zipcodes_agg, boroughs, how="left", on="ZIP")
print(zipcodes_agg.head())
print(zipcodes_agg["LNG"].unique())
#print(len(zipcodes_aggr))

#abc = zipcodes_agg["LAT"].notnull()
#print(len(abc))
#zipcodes_agg.loc[116,'LAT']="40.6933"
#zipcodes_agg.loc[116,'LNG']="-73.9925"



map = folium.Map(location=[40.693943, -73.985880], default_zoom_start=15)
map.choropleth(geo_data="data/nyc-zip-code-tabulation-areas-polygons.geojson",
             data=zipcodes_agg, # my dataset
             columns=["ZIP", "SALE PRICE"], # zip code is here for matching the geojson zipcode, sales price is the column that changes the color of zipcode areas
             key_on='feature.properties.postalCode', # this path contains zipcodes in str type, this zipcodes should match with our ZIP CODE column
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2,
             legend_name='SALE PRICE')

marker_cluster = MarkerCluster().add_to(map)

for i in range(zipcodes_agg.shape[0]):
    location = [zipcodes_agg["LAT"][i],zipcodes_agg["LNG"][i]]
    tooltip = "Zipcode:{}<br> Borough: {}<br> Click for more".format(zipcodes_agg["ZIP"][i], zipcodes_agg['BOROUGH'][i])

    folium.Marker(location, # adding more details to the popup screen using HTML
                  popup="""
                  <i>Mean sales price: </i> <br> <b>${}</b> <br>
                  <i>mean total units: </i><b><br>{}</b><br>
                  <i>mean square feet: </i><b><br>{}</b><br>""".format(
                    round(zipcodes_agg['SALE PRICE'][i],2),
                    round(zipcodes_agg['TOTAL UNITS'][i],2),
                    round(zipcodes_agg['GROSS SQUARE FEET'][i],2)),
                  tooltip=tooltip).add_to(marker_cluster)
map
map.save("data/nyc_map.html")


C:\Users\Dome\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Dome\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Dome\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWa

     ZIP       BLOCK          LOT  ZIP CODE  RESIDENTIAL UNITS  \
0  10001  769.426471   550.274510   10001.0           7.303922   
1  10002  328.079268   359.317073   10002.0           2.106707   
2  10003  672.133005   531.520936   10003.0           1.391626   
3  10004   18.894737  1776.642105   10004.0           1.221053   
4  10005   32.226131  1234.095477   10005.0           4.929648   

   COMMERCIAL UNITS  TOTAL UNITS  LAND SQUARE FEET  GROSS SQUARE FEET  \
0          1.710784     9.088235        308.899743        2461.706351   
1          0.310976     2.484756         49.807904         182.647015   
2          0.190887     1.619458         30.834529         173.948854   
3          0.031579     1.252632         55.184382        1284.870004   
4          0.190955     5.135678         54.844983        1403.926327   

    YEAR BUILT  TAX CLASS AT TIME OF SALE    SALE PRICE        LAT  \
0  1324.975490                   2.519608  1.245806e+07  40.750633   
1  1730.283537          

ValueError: Location values cannot contain NaNs.